# **Semiconductor_Band Gap Prediction (Regression)**

# **1. Data Loading**




In [ ]:
import kagglehub
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# --- 1. Download & Load Data ---
# Download dataset
path = kagglehub.dataset_download("allanwandia/material-science")

# Find the CSV file in the download folder
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
full_path = os.path.join(path, csv_files[0])

df = pd.read_csv(full_path)

# **2. Checking data information**




**Basic command for checking data information**



```
df.info()
df.head()
df.tail()
df.describe()
df.columns

In [ ]:
df.info()

**Check for missing values**

In [ ]:
# Check for missing values
print('Missing values in each column:')
print(df.isnull().sum())



**If any numeric columns have missing values, fill them with the median**

In [ ]:
# Basic cleaning: if any numeric columns have missing values, fill them with the median

numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

# For any categorical columns with missing values, fill with 'Unknown'
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna('Unknown', inplace=True)

# Final shape after cleaning
print('Data shape after cleaning:', df.shape)

In [ ]:
df.tail()

In [ ]:
df.describe()

**Histogram for numeric columns**

In [ ]:
## Histogram for numeric columns
numeric_df = df.select_dtypes(include=[np.number])
for col in numeric_df.columns:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[col], kde=True)
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

**Correlation Heatmap**

In [ ]:
## Correlation Heatmap (if there are 4 or more numeric columns)
if numeric_df.shape[1] >= 4:
    plt.figure(figsize=(10, 8))
    corr = numeric_df.corr()
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
    plt.title('Correlation Heatmap of Numeric Features')
    plt.tight_layout()
    plt.show()


**Pair Plot for variables**

In [ ]:
sns.pairplot(numeric_df)
plt.suptitle('Pair Plot for Numeric Features', y=1.02)
plt.show()

# **3. Modeling**

**Define target variable and select feature set**


In [ ]:
# Target: Convert True/False to 1/0
df_class = df.copy()
df_class['target'] = df_class['band_gap']
df_class['target'].head()

**Features: Physical properties**

In [ ]:
# Features: Physical properties
features = ['density', 'formation_energy_per_atom', 'volume', 'n_elements']
X = df_class[features]
y = df_class['target']

**Split Data**

In [ ]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Train Single Model**

In [ ]:
# Evaluate ---
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np
import joblib
model =  RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Display training and testing scores
print(f'Training Score: {model.score(X_train, y_train):.4f}')
print(f'Testing Score: {model.score(X_test, y_test):.4f}')

# Evaluate RMSE for the best model
model_pred = model.predict(X_test)
model_mae = np.sqrt(mean_absolute_error(y_test, model_pred))
print(f'Best Model MAE: {model_mae:.4f}')



**Multiple Modeling**

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# 1. Setup the 6 Contenders
models = {
    "Linear Regression": LinearRegression(),
    "K-Nearest Neighbors": make_pipeline(StandardScaler(), KNeighborsRegressor()),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting (sklearn)": GradientBoostingRegressor(random_state=42),
    "Neural Network": make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(100,50), max_iter=1000, random_state=42)),
    "XGBoost": XGBRegressor(random_state=42)
}

# List to store results for sorting
results_list = []

print("Training models... please wait.")

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Store in list
    results_list.append({
        "Model": name,
        "R2": r2,
        "MAE": mae
    })

# --- SORTING LOGIC ---
# Sort by R2 Score in Descending order (Highest R2 is best)
# If you prefer MAE, change key to x['MAE'] and reverse=False
sorted_results = sorted(results_list, key=lambda x: x['R2'], reverse=True)

# --- PRINTING ---
print("\n" + "="*55)
print(f"{'Model Name':<30} | {'R2 Score':<10} | {'MAE (eV)':<10}")
print("-" * 55)

for res in sorted_results:
    print(f"{res['Model']:<30} | {res['R2']:.4f}     | {res['MAE']:.4f}")
print("="*55)
print(f"🏆 Best Model: {sorted_results[0]['Model']}")

# **7. Save the best model**

In [ ]:
import joblib

# 1. Identify the Winner
# Grab the name of the model in the first row (since it's sorted by R2 descending)
best_model_name = sorted_results[0]['Model']
print(f"\n🏆 The Best Model is: {best_model_name}")

# 2. Retrieve the Trained Object
# Look up the actual model object from your original 'models' dictionary
best_model = models[best_model_name]

# 3. Save to Disk
filename = 'best_concrete_model_BG.pkl'
joblib.dump(best_model, filename)

print(f"✅ Successfully saved {best_model_name} to '{filename}'")

In [ ]:
import matplotlib.pyplot as plt

# 1. Get predictions for BOTH sets
train_preds = best_model.predict(X_train)
test_preds = best_model.predict(X_test)

plt.figure(figsize=(9, 9))

# 2. Plot Training Data
# We use a lower 'alpha' (transparency) because there are usually many more training points
plt.scatter(y_train, train_preds, color='tomato', alpha=0.3, label='Training Set (Memorized)')

# 3. Plot Test Data
plt.scatter(y_test, test_preds, color='skyblue', alpha=0.2, label='Test Set (New Data)')

# 4. Draw the "Perfect Prediction" Line
min_val = min(y.min(), train_preds.min()) # Find the lowest value in the whole dataset
max_val = max(y.max(), train_preds.max()) # Find the highest value
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=3, label='Perfect Match')

plt.title("Overfitting Check: Train vs. Test")
plt.xlabel("Actual Strength (MPa)")
plt.ylabel("Predicted Strength (MPa)")
plt.legend()
plt.grid(True)
plt.show()

# **6. Feature importances**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'best_model' now holds the trained RandomForestRegressor
# If not, you can load it: best_model = joblib.load('best_materials_model.pkl')
# For this example, let's assume you access the trained object from the loop.

# --- 1. Get the Importance Scores ---
# This is a key feature of all tree-based models!
importances = best_model.feature_importances_

# 2. Create a clean table
# Features list needs to be defined: features = ['density', 'formation_energy_per_atom', 'volume', 'n_elements']
features = ['density', 'formation_energy_per_atom', 'volume', 'n_elements'] # Assuming your list

feature_table = pd.DataFrame({
    'Feature': features,
    'Importance': importances
})

# 3. Sort and print
feature_table = feature_table.sort_values(by='Importance', ascending=False)
print("\n--- Feature Importance for Random Forest Regressor ---")
print(feature_table)

# 4. Visualize it!
plt.figure(figsize=(8, 4))
plt.barh(feature_table['Feature'], feature_table['Importance'], color='skyblue')
plt.xlabel('Importance Score (Total = 1.0)')
plt.title('Feature Impact on Band Gap Prediction')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
import pandas as pd
import joblib

# 1. LOAD the model
loaded_model = joblib.load('best_concrete_model_BG.pkl')
# Check the feature names expected by the model
# (Run this line to see the correct names/order)
#print(f"Feature names expected: {loaded_model.feature_names_in_}")

# 2. Create a DataFrame with the correct column names
# ** IMPORTANT: The names and order must match Step 1 exactly **
new_material_df = pd.DataFrame([[5.0, -1.2, 100, 3]],
                               columns=loaded_model.feature_names_in_)

# 3. Make prediction (The warning will be gone)
prediction = loaded_model.predict(new_material_df)
print(f"Predicted band gap: {prediction[0]:.4f}")
